В этой части я займусь визуализацией знаний, который хранятся и обрабатываются в сверточной нейронной сети



1.  карт признаков, выделенных слоями
2.  фильтров слоев
3.  тепловых карт









# ЧАСТЬ 1: ВИЗУАЛИЗАЦИЯ ПРОМЕЖУТОЧНЫХ АКТИВАЦИЙ (КАРТ ПРИЗНАКОВ)

Загрузка ранее натренированной модели

In [ ]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/StudyingKeras/Dogs_vs_Cats/weights/cats_vs_dogs_small_1.h5')
model.summary()

Выберем изображение, которое сеть не видела

In [ ]:
img_path = '/content/drive/My Drive/StudyingKeras/Dogs_vs_Cats/partial_data/test/cats/cat.1700.jpg'

Визуализация изображения

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

img = image.load_img(img_path, target_size=(150,150))
# преобразуем картинки в 4мерный тензор
img_tensor = image.img_to_array(img)
# добавление еще одной оси спереди тензора
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255

# форма тензора - (1, 150, 150, 3)
print(img_tensor.shape)

plt.imshow(img_tensor[0])
plt.show()

Создадим свою модель, обладающую несколькими выходами

In [ ]:
# класс Model() принимает два параметра:
# 1) Входной (ые) тензор
# 2) Выходной (ые) тензор
# отличительная черта от Sequantial() - может иметь несколько выходов
from keras.models import Model

# извлечение вывода верхних восьми слоев
layer_outputs = [layer.output for layer in model.layers[:8]]
# входные данные такие же, а выходные - новые
activation_model = Model(inputs=model.input, outputs=layer_outputs)

Запуск модели в режиме прогнозирования

In [ ]:
activations = activation_model.predict(img_tensor)

# возьмем активацию первого слоя
first_layer_activation = activations[0]
print(first_layer_activation.shape)

Визуализация четвертого канала предсказаний первого слоя модели

In [ ]:
plt.clf()
plt.imshow(first_layer_activation[0, :, :, 4], cmap='viridis')
plt.show()

Теперь создадим полную визуализацию всех 32 каналов на каждой из 8ми карт активации

Чем ниже картинки - тем глубже слой сети

Чем глубже слой сети - тем более абстрактные у него активации

Например, первый слой (верхняя картинка) может находить какие-то прямые, линии и т.д, а последний слой (нижняя картинка) активируется такими вещами как "ухо" или "глаз" поэтому и пиксели на изображении там, где уши или глаза

По сути, человек воспринимает мир так же. Если вас попросить нарисовать велосипед, то вы нарисуете пару палок и колеса, а не все делати механизмов. То 
есть вы запоминаете только самые важные части. 

In [ ]:
def visualise_features_maps():
  plt.clf()
  layer_names = []
  # извлекаем имена слоев, чтобы потом отобразить их на рисунках
  for layer in model.layers:
    layer_names.append(layer.name)
  # так как карта признаков извлекает 32 * k признаков, то берем число, которому
  # 32 кратно просто для красоты
  images_per_row = 16
  for layer_name, layer_activation in zip(layer_names, activations):
    # карта признаков имеет форму (1, size, size, n_features)
    # берем только последнюю ось тензора
    n_features = layer_activation.shape[-1]
    # берем только первую ось тензора
    size = layer_activation.shape[1]

    # количество строчек в таблице отображений
    n_cols = n_features // images_per_row
    # сама таблица инициализируется нулями
    display_grid = np.zeros((size * n_cols, size * images_per_row))

    # обычный вложенный цикл, как в двумерной матрице
    for col in range(n_cols):
      for row in range(images_per_row):
        # это - каждый маленький квадрат в таблице отображений
        # берется каждый признак из последней оси активации (которая кратна 32)
        channel_image = layer_activation[0, :, :, col * images_per_row + row]
        # обработка изображения для приемлимого вида
        channel_image -= channel_image.mean()
        channel_image /= channel_image.std()
        channel_image *= 64
        channel_image += 128
        channel_image = np.clip(channel_image, 0, 255).astype('uint8')
        display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image
    
    # size - размер каждого квадратного изображения без уменьшения
    scale = 1. / size
    # уменьшаем размер общей сетки отображений, чтобы все картинки поместились на ней
    plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

visualise_features_maps()

# ЧАСТЬ 2: ВИЗУАЛИЗАЦИЯ ФИЛЬТРОВ СЛОЕВ

Надо отобразить шаблон, за который отвечает каждый фильтр, для этого:


1.   Создадим функцию потерь, максимизирующую значение данного фильтра данного слов
2.   С помощью СГС настроим ВХОДНОЕ изображение так, чтобы оно максимизировало данную функцию активации



Определение тензора потерь для визуализации фильтра

In [ ]:
from keras.applications import VGG16
from keras import backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

model = VGG16(
    weights='imagenet',
    include_top=False
)

layer_name = 'block3_conv1'
filter_index = 0

layer_output = model.get_layer(layer_name).output
loss = K.mean(layer_output[:, :, :, filter_index])

Получение градиента потерь относительно входа модели

In [ ]:
import tensorflow as tf
# gradients возвращает список тензоров, сохраняется только первый
grads = K.gradients(loss, model.input)[0]

Нормализация градиентного тензора


In [ ]:
# чтобы гарантировать то, что величина изменений во входном изображении
# будет находиться в одном диапозоне, можно разделить градиентный тензор
# на квадратный корень из усредненных квадратов значений в тензоре
# в конце добавляем очень маленькое число, чтобы предотвратить деление на ноль
# если вдруг дробь будет нулевая
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5) 

Вычисление тензора потерь и тензора градиента для заданного входного изображения

In [ ]:
# мы создаем вручную свою функцию, которая затем может быть использована в Keras
# документация - https://www.tensorflow.org/api_docs/python/tf/keras/backend/function
# принимает массив входных тензоров
# возвращает массив из двух выходных тензоров
iterate =  K.function([model.input], [loss, grads])
loss_value, grads_value = iterate([np.zeros((1, 150, 150, 3))])

Максимизация потерь стохастическим градиентным спуском

In [ ]:
# начальное изображение с черно-белым шумом
# создается случайно (через рандомные floatы)
input_image_data = np.random.random((1, 150, 150, 3)) * 20 + 128
# величина каждого изменения градиента
step = 1
# 40 шагов градиентного восхождения
for i in range(40):
  # вычисление значений потерь и градиента
  loss_value, grads_value = iterate([input_image_data])
  # корректировка входного изображения в направлении максимизации потерь
  input_image_data += grads_value * step

Теперь тензор необходимо превратить в изображение

In [ ]:
def deprocess_image(x):
  # нормализация: получается тензор со средним значением 0 и стандартным откло
  # нением 0.1
  x -= x.mean()
  x /= (x.std() + 1e-5)
  x *= 0.1
  x += 0.5
  # ограничиваем значения диапозоном от 0 до 1 
  # Функция clip() ограниченичивает элементы массива указанным интервалом
  # допустимых значений. Например, если указать интервал [5, 10], то все 
  # значения в массиве, которые меньше 5 примут значение равное 5, а все 
  # значения больше 10, будут равны 10.
  x = np.clip(x, 0, 1)
  x *= 255
  # преобразование в массив значений RGB
  x = np.clip(x, 0, 255).astype('uint8')
  return x

Объединю все выше показанное в одну функию

Функция генерирует изображение, которое представляет фильтр


In [ ]:
# функция принимает имя слоя и индекс фильтра
# функция возвращает тензор с допустимым изображением, представляющим 
# собой шаблон, который максимизирует активацию данного фильтра
def generate_pattern(layer_name, filter_index, size=150):
  layer_output = model.get_layer(layer_name).output
  loss = K.mean(layer_output[:, :, :, filter_index])
  grads = K.gradients(loss, model.input)[0]
  grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5) 
  iterate =  K.function([model.input], [loss, grads])
  input_image_data = np.random.random((1, 150, 150, 3)) * 20 + 128
  step = 1
  for i in range(40):
    loss_value, grads_value = iterate([input_image_data])
    input_image_data += grads_value * step
  img = input_image_data[0]
  return deprocess_image(img)

In [ ]:
generated_image = generate_pattern('block3_conv1', 0)
plt.imshow(generated_image)

# похоже, что фильтр с индексом 0 в слое отвечает за узор "в горошек"

Вывод откликов всех фильтров в указанном слое

In [ ]:
layer_name = 'block3_conv1'
size = 150

plt.clf()

for i in range(8):
  for j in range(8):
    filter_img = generate_pattern(layer_name, i + (j * 8), size=size)
    plt.figure(i + j)
    imgplot = plt.imshow(filter_img)
  
plt.show()

# ЧАСТЬ 3: ВИЗУАЛИЗАЦИЯ ТЕПЛОВЫХ КАРТ АКТИВАЦИИ СЛОЕВ

Тепловая карта признаков - карта признаков, которая обозначает "как интенсивно входное изображение активирует класс"

Определяет насколько важно каждое местоположение на картинке для каждого класса

Производится взвешивание признаков в пространственной карте "как интенсивно входное изображение активирует каналы" по категории "насколько важен каждый
канал для класса"

In [ ]:
from keras.applications.vgg16 import VGG16

# полносвязный классификатор НЕ удаляется, как в прошлых случаях
model = VGG16(weights='imagenet')
model._make_predict_function()

Предобработка изображения 

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

img_path = '/content/drive/My Drive/StudyingKeras/FeaturesVisualisation/elephants.PNG'

img = image.load_img(img_path, target_size=(224, 224))
# переводим изображение в массив numpy (224, 224, 3)
x = image.img_to_array(img)
# увеличиваем размерность на 1 (1, 224, 224, 3)
x = np.expand_dims(x, axis=0)
# нормализация пакетов цвета
x = preprocess_input(x)

Предсказание на изображении

In [ ]:
preds = model.predict(x)
# decode_predictions возвращает массив массивов классов, которые были распознаны 
# на данной фотографии с наибольшей вероятностью (класс, описание, вероятность)
# лучшие предсказания: африканский слон, кабан-секач, индийский слон
print(f"Predicted here: \n {decode_predictions(preds, top=3)}")

Выведем класс с наибольшей вероятностью

In [ ]:
print(f'Class number is {np.argmax(preds[0])}')

Реализация алгоритма Grad_CAM

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

import keras.backend as K
# выведем часть иображения, которая наиболее близко напоминает африканского слона
# [:, 386] выведет 386 столбец из дмумерного тензора, например:
# line = [[1 2 3]
#         [4 5 6]]
# line[:,2] = [3 6]

african_elephant_output = model.output[:, 386]
# выходная карта последнего сверточного слоя
last_conv_layer_map = model.get_layer('block5_conv3')
# градиент класса "африканский слон" для выходной карты признаков последнего слоя
grads = K.gradients(african_elephant_output, last_conv_layer_map.output)[0]
# вектор формы (512,) каждый элемент которого определяет интенсивность
# градиента для заданного канала в карте признаков
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# function - очень хитрая вещь
# мы в параметрах указываем переменные КОТОРЫЕ УЖЕ СОЗДАНЫ
# она понимает СВЯЗЬ между ними, и какие действия нужно сделать
# поэтому если вместо model мы создадим model_1 и поместим в функцию,
# то она для НОВОЙ модели посчитает градиенты и выход последнего слоя и возвратит
iterate = K.function(inputs=[model.input], outputs=[pooled_grads, last_conv_layer_map.output[0]])

pooled_grads_value, conv_layer_output_value = iterate([x])

for i in range(512):
  # умножение каждого канала на "важность" этого канада для класса "афр.слон"
  conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# среднее для каналов признаков - и есть тепловая карта активации
heatmap = np.mean(conv_layer_output_value, axis=-1)

# нормализация тепловой карты
# для примера: np.maximum([[0, 1, -5][-9, 3, 5]) = [0, 3, 5]
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.clf()
plt.matshow(heatmap)

Наложим тепловую карту на исходное изображение и выведем его

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
# преобразование тепловой карты в RGB
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
img = heatmap * 0.6 + img
cv2_imshow(img)

# мы видим, что уши слоненка сильно активированы
# видимо, именно по ним сеть и приняла решение, что на фото - слоны